In [15]:
# Load Disease data of all states

import pandas
import numpy as np
import os                                                                   
import glob
import warnings

warnings.filterwarnings("ignore")

state = 1
allstatescopd = None
ll = glob.glob('*.CSV')

for cl in ll:
    
    data = pandas.read_csv(cl, delimiter = ',')

    cvd = data[data['cause_name'] == 'Chronic obstructive pulmonary disease']
    sname = data['location_name'][0]
    csvfname = sname.lower().replace(' ', '') + 'copd'
    cvdn = cvd[cvd['location_name'] != sname]
    cvdnb = cvdn[cvdn['sex'] == 'Both']
    cvdnb['key_id'] = sname.lower() + ', ' + cvdnb['location_name'].str.lower().str.extract(r'^([\S ]+)', expand= True)
    
    cvdnb80 = cvdnb[cvdnb['year_id'] == 1980]
    cvdnb81 = cvdnb[cvdnb['year_id'] == 1981]
    
    cvdnb81_n = cvdnb81[['mx', 'key_id']]
    cvdnb81_n = cvdnb81_n.rename(columns={'mx': 'n_mx'})
    cvdnb80j = cvdnb80.join(cvdnb81_n.set_index('key_id'), on='key_id', how='inner')
    
    cvdlist = []
    
    for i in np.arange(1981, 2014, 1):
        
        cvdnb_p = cvdnb[cvdnb['year_id'] == i - 1]
        cvdnb_c = cvdnb[cvdnb['year_id'] == i]
        cvdnb_n = cvdnb[cvdnb['year_id'] == i + 1]
        
        cvdnb_p_ = cvdnb_p[['mx', 'key_id']]
        cvdnb_p_ = cvdnb_p_.rename(columns={'mx': 'p_mx'})
        cvdnb_cj = cvdnb_c.join(cvdnb_p_.set_index('key_id'), on='key_id', how='inner')

        cvdnb_n_n = cvdnb_n[['mx', 'key_id']]
        cvdnb_n_n = cvdnb_n_n.rename(columns={'mx': 'n_mx'})
        cvdnb_cj = cvdnb_cj.join(cvdnb_n_n.set_index('key_id'), on='key_id', how='inner')
        
        cvdlist.append(cvdnb_cj)
    
    cvdjoined = cvdnb80j.append(cvdlist, ignore_index=True, sort=False)
    cvdjoined['key_id'] = cvdjoined['year_id'].apply(str) + ", " + cvdjoined['key_id']
    
    if state == 1:
        allstatescopd = cvdjoined
        state += 1
    else:
        allstatescopd = allstatescopd.append(cvdjoined, ignore_index=True, sort=False)
           
cvddata = allstatescopd[['measure_id', 'measure_name', 'location_id',
       'location_name', 'FIPS', 'cause_id', 'cause_name', 'sex_id', 'sex',
       'age_id', 'age_name', 'year_id', 'metric', 'mx', 'upper', 'lower',
       'key_id', 'n_mx', 'p_mx']]

cvddata.to_csv('allstatescopddata.csv', sep=',')

In [16]:
# Load Air Quality data

aird = pandas.read_csv('aiq.csv', delimiter=',')
airq = aird[aird['Year'] != 'Year']
airq['key_id'] = airq['Year'].apply(str) + ", " + airq['State'].str.lower() + ", " + airq['County'].str.lower() + " county"
airq.to_csv('airqualitydata.csv', sep=',')

In [17]:
# Merge Disease data and Air Quality data

airn = pandas.read_csv('airqualitydata.csv', delimiter=',')
air = airn[['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days SO2', 'Days PM2.5', 'Days PM10', 'key_id']]
copdn = pandas.read_csv('allstatescopddata.csv', delimiter=',')
copd = copdn[['measure_id', 'measure_name', 'location_id',
       'location_name', 'FIPS', 'cause_id', 'cause_name', 'sex_id', 'sex',
       'age_id', 'age_name', 'year_id', 'metric', 'mx', 'upper', 'lower',
       'key_id', 'n_mx', 'p_mx']]
aircopd = air.join(copd.set_index('key_id'), on='key_id', how='inner')
aircopd['class'] = np.where((aircopd['mx'] < aircopd['n_mx']), 1, 0)
aircopd.reset_index(drop=True, inplace=True)
aircopd.to_csv('aircopd.csv', sep=',')

In [18]:
data = pandas.read_csv('aircopd.csv', delimiter=',')
data

,Unnamed: 0,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,...,age_id,age_name,year_id,metric,mx,upper,lower,n_mx,p_mx,class
0,0,Alabama,Autauga,1980,179,122,35,18,4,0,...,27,Age-standardized,1980,Rate,41.293067,45.602668,37.347579,41.794329,NaN,1
1,1,Alabama,Colbert,1980,274,127,45,63,39,0,...,27,Age-standardized,1980,Rate,32.257610,35.276436,29.249302,33.981366,NaN,1
2,2,Alabama,Jackson,1980,366,85,110,92,79,0,...,27,Age-standardized,1980,Rate,36.673665,40.137170,33.519399,36.691882,NaN,1
3,3,Alabama,Jefferson,1980,343,171,109,37,19,7,...,27,Age-standardized,1980,Rate,40.051148,42.088048,37.820208,40.620596,NaN,1
4,4,Alabama,Lauderdale,1980,274,120,58,77,19,0,...,27,Age-standardized,1980,Rate,33.300949,35.956983,30.585845,33.568443,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29900,29900,Wyoming,Sublette,2013,365,282,83,0,0,0,...,27,Age-standardized,2013,Rate,57.837035,64.803660,51.462355,57.902727,58.619756,1
29901,29901,Wyoming,Sweetwater,2013,365,252,109,3,0,0,...,27,Age-standardized,2013,Rate,66.766262,71.767134,61.914160,67.906988,67.930619,1
29902,29902,Wyoming,Teton,2013,365,310,55,0,0,0,...,27,Age-standardized,2013,Rate,29.744798,33.200315,26.231828,29.406834,29.821489,0
29903,29903,Wyoming,Uinta,2013,365,312,53,0,0,0,...,27,Age-standardized,2013,Rate,71.299333,77.319022,65.447915,72.153657,69.922598,1
